In [1]:
from ipywidgets import widgets
from IPython.display import display
import numpy as np
import bqplot as bq
import threading
import numpy

In [2]:
# slider
silder_amplitude = widgets.FloatSlider(min=0, max=5.0, step=0.1, value=1)
slider_frequency = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.5)

# label
label_current_thread = widgets.Label()
label_thread_list = widgets.Label()

# button 
button_stop = widgets.Button(
    description = 'stop',
    icon = 'fa-stop',
    button_style = 'warning',
    layout = widgets.Layout(width='100px')
)
def button_stop_click(self):
    global thread_status
    thread_status = False
button_stop.on_click(button_stop_click)

button_check_thread = widgets.Button(
    description = 'check thread',
    button_style = '',
    layout = widgets.Layout(width='100px')
)
def button_check_thread_click(self):
    # print if work() still running
    work_count = 0
    name='work'
    for thread_object in threading.enumerate():
        if thread_object.name == name:
            work_count+=1
    if work_count > 0:
        label_thread_list.value=f'{work_count} thread with name {name} still exist. Consider use exit() and restart kernel'
    else:
        label_thread_list.value=f'No thread with name {name} exist'
button_check_thread.on_click(button_check_thread_click)

In [3]:
#scale
x_sc = bq.LinearScale()
y_sc = bq.LinearScale()

#axis
x_ax = bq.Axis(
    label = 'x(t)',
    scale = x_sc
)
y_ax = bq.Axis(
    label = 'y(t)',
    scale = y_sc,
    orientation = 'vertical'
)

# line
line = bq.Lines(
    scales = {
        'x': x_sc,'y': y_sc
    }
)

# figure
fig = bq.Figure(
    layout = widgets.Layout(width='500px', height='300px'),
    axes = [x_ax, y_ax],
    marks = [line],
    fig_margin = dict(top=10, bottom=40, left=50, right=10),   
)

In [4]:
def make_x(start=0,stop=2*np.pi,num=100,endpoint=True):
    return np.linspace(start=start,stop=stop,num=num,endpoint=endpoint)

In [5]:
def make_sin_wave(x, amplitude, frequency):
    return amplitude * np.sin(2 * np.pi * frequency * x)

In [6]:
def work():
    global thread_status
    
    if thread_status:
        label_current_thread.value = 'Thread On'
    
    current_state = []
    
    while thread_status:
        if current_state != [
            silder_amplitude.value,
            slider_frequency.value
        ]:

            current_state = [
                silder_amplitude.value,
                slider_frequency.value
            ]

            line.x = make_x()
            line.y = make_sin_wave(line.x,silder_amplitude.value, slider_frequency.value)
    
    if not thread_status:
        label_current_thread.value = 'Thread Off'

In [7]:
# restart flag
thread_status = False
thread_status = True

# create thread
thread_work = threading.Thread(target=work)
thread_work.name = 'work'

# start thread
thread_work.start()

In [8]:
# wrapper
wrapper = widgets.VBox(
    children = (
        fig,
        silder_amplitude,
        slider_frequency,
        label_current_thread,
        button_stop,
    )
)
display(wrapper)

In [9]:
display(button_check_thread)
display(label_thread_list)

Button(description='check thread', layout=Layout(width='100px'), style=ButtonStyle())

Label(value='')